In [1]:
# !pip install requests beautifulsoup4 lxml selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 25.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.4
    Uninstalling websocket-client-1.6.4:
      Successfully uninstalled websocket-client-1.6.4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13
  Attempting uninstall: typing_extensions
    Found existing installation: typin

In [2]:
# -*- coding: utf-8 -*-
"""
CE.LA ISMS 페이지(https://www.cela.kr/ISMS_1_1) 크롤러
- 정적: requests + BeautifulSoup
- 동적: Selenium 백업 + '접기/펼치기/상세' 등 모든 토글 열기
- 표/문단/목록/링크를 구조화하여 CSV/JSON 저장
"""

import os
import time
import json
import csv
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional

import requests
from bs4 import BeautifulSoup

# ===== 설정 =====
TARGET_URL = "https://www.cela.kr/ISMS_1_1"
OUT_DIR = "../data"
TIMEOUT = 20
RETRY = 3
SLEEP_SEC = 0.8
USE_SELENIUM_FALLBACK = True   # 정적 파싱이 비면 Selenium로 재시도(+토글 전부 펼치기)

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/134.0.0.0 Safari/537.36"
    )
}

@dataclass
class ContentRow:
    section_order: int
    section_title: str
    content_type: str   # "paragraph" | "list" | "table" | "link"
    text: str           # 표는 TSV 문자열
    href: Optional[str] = None


def safe_mkdir(path: str):
    os.makedirs(path, exist_ok=True)


def fetch_html(url: str, headers: Dict[str, str], timeout: int = TIMEOUT) -> str:
    last_exc = None
    for _ in range(RETRY):
        try:
            resp = requests.get(url, headers=headers, timeout=timeout)
            if 200 <= resp.status_code < 300:
                return resp.text
        except Exception as e:
            last_exc = e
        time.sleep(0.8)
    if last_exc:
        raise last_exc
    raise RuntimeError("Failed to fetch HTML")


def load_with_selenium_and_expand(url: str) -> str:
    """
    Selenium로 페이지 로드 후, 접기/펼치기/상세 등 숨겨진 영역을 모두 펼친 뒤 HTML 반환.
    - pip install selenium webdriver-manager
    - 로컬/서버에 Chrome/Chromium 필요
    """
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import (
        StaleElementReferenceException,
        ElementClickInterceptedException,
        ElementNotInteractableException,
        TimeoutException,
        WebDriverException,
    )

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--lang=ko-KR")
    options.add_argument(f"--user-agent={HEADERS['User-Agent']}")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=options)
    wait = WebDriverWait(driver, 15)
    try:
        driver.get(url)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(1.2)

        def click_safe(el):
            try:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                time.sleep(0.15)
                try:
                    el.click()
                except (ElementClickInterceptedException, ElementNotInteractableException):
                    driver.execute_script("arguments[0].click();", el)
                time.sleep(0.2)
                return True
            except WebDriverException:
                return False

        def expand_round() -> int:
            """한 라운드에서 열린 요소 수(클릭 성공 수) 반환"""
            opened = 0

            # 1) details/summary
            summaries = driver.find_elements(By.CSS_SELECTOR, "details:not([open]) > summary")
            for s in summaries:
                if click_safe(s):
                    opened += 1

            # 2) aria-expanded=false → true로 전환될 토글
            toggles = driver.find_elements(By.CSS_SELECTOR, "[aria-expanded='false'], [aria-pressed='false']")
            for t in toggles:
                if click_safe(t):
                    opened += 1

            # 3) '펼치기/접기/상세/자세히/더보기' 텍스트가 있는 버튼/링크/요소
            keywords = ("펼치기", "접기", "상세", "자세히", "더보기")
            candidates = driver.find_elements(By.CSS_SELECTOR, "button, a, [role='button'], .toggle, .accordion, .collapse, .expander")
            for c in candidates:
                try:
                    txt = (c.text or "").strip()
                except StaleElementReferenceException:
                    continue
                if any(k in txt for k in keywords):
                    if click_safe(c):
                        opened += 1

            # 4) 흔한 아코디언 트리거 클래스 추가 탐색
            more_candidates = driver.find_elements(By.XPATH,
                "//*[contains(@class,'accordion') or contains(@class,'toggle') or contains(@class,'collapse') or contains(@class,'expand')]"
            )
            for m in more_candidates:
                try:
                    txt2 = (m.text or "").strip()
                except StaleElementReferenceException:
                    continue
                if any(k in txt2 for k in keywords):
                    if click_safe(m):
                        opened += 1

            return opened

        # 열릴 게 없을 때까지 반복 (상한선으로 안전장치)
        MAX_ROUNDS = 8
        for _ in range(MAX_ROUNDS):
            changed = expand_round()
            if changed == 0:
                break
            time.sleep(0.6)

        # 모든 콘텐츠가 DOM에 노출되었을 가능성이 높으니 페이지 소스 추출
        time.sleep(0.6)
        html = driver.page_source
        return html

    finally:
        driver.quit()


# ===== 파서 =====
def normalize_text(el) -> str:
    return " ".join(el.get_text(" ", strip=True).split())


def extract_tables(soup: BeautifulSoup) -> List[str]:
    tables_tsv: List[str] = []
    for table in soup.select("table"):
        rows = []
        for tr in table.select("tr"):
            cells = tr.select("th, td")
            row = []
            for c in cells:
                txt = " ".join(c.get_text(" ", strip=True).split())
                row.append(txt)
            rows.append("\t".join(row))
        if rows:
            tables_tsv.append("\n".join(rows))
    return tables_tsv


def guess_sections(soup: BeautifulSoup) -> List[Dict]:
    headers = soup.select("h1, h2, h3, h4")
    sections = []
    for h in headers:
        title = normalize_text(h)
        content_nodes = []
        for sib in h.find_all_next():
            if sib == h:
                continue
            if sib.name in ["h1", "h2", "h3", "h4"]:
                break
            if sib.name in ["script", "style", "noscript"]:
                continue
            content_nodes.append(sib)
        sections.append({"title": title, "nodes": content_nodes})
    if not sections:
        body = soup.body or soup
        sections = [{"title": "본문", "nodes": list(body.children)}]
    return sections


def parse_section(title: str, nodes: List) -> List[ContentRow]:
    rows: List[ContentRow] = []
    order = 0

    temp_soup = BeautifulSoup("".join(str(n) for n in nodes), "html.parser")

    # 표
    for tsv in extract_tables(temp_soup):
        rows.append(ContentRow(order, title, "table", tsv))
        order += 1

    # 문단
    for p in temp_soup.select("p"):
        txt = normalize_text(p)
        if txt:
            rows.append(ContentRow(order, title, "paragraph", txt))
            order += 1

    # 목록 (리스트 내부 모든 항목 수집: 중첩 포함)
    for lst in temp_soup.select("ul, ol"):
        items = []
        for li in lst.select("li"):   # ← 여기! 모든 li 수집
            t = normalize_text(li)
            if t:
                items.append(t)
        if items:
            rows.append(ContentRow(order, title, "list", "\n".join(items)))
            order += 1

    # 링크
    for a in temp_soup.select("a[href]"):
        href = a.get("href", "").strip()
        txt = normalize_text(a)
        if href and not href.startswith("javascript:"):
            rows.append(ContentRow(order, title, "link", txt, href=href))
            order += 1

    return rows


def crawl_cela_isms(url: str) -> List[ContentRow]:
    html = fetch_html(url, HEADERS)
    soup = BeautifulSoup(html, "html.parser")

    # 본문 텍스트가 너무 적으면 동적일 가능성 → Selenium로 전체 펼치기
    if len(soup.get_text(strip=True)) < 600 and USE_SELENIUM_FALLBACK:
        html = load_with_selenium_and_expand(url)
        soup = BeautifulSoup(html, "html.parser")

    sections = guess_sections(soup)

    all_rows: List[ContentRow] = []
    for i, sec in enumerate(sections, start=1):
        sec_title = sec["title"]
        sec_rows = parse_section(sec_title, sec["nodes"])
        for r in sec_rows:
            r.section_order = i
        all_rows.extend(sec_rows)
        time.sleep(SLEEP_SEC)

    return all_rows


def save_as_csv(rows: List[ContentRow], path: str):
    with open(path, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["section_order", "section_title", "content_type", "text", "href"])
        for r in rows:
            writer.writerow([r.section_order, r.section_title, r.content_type, r.text, r.href or ""])


def save_as_json(rows: List[ContentRow], path: str):
    with open(path, "w", encoding="utf-8") as f:
        json.dump([asdict(r) for r in rows], f, ensure_ascii=False, indent=2)


In [3]:
safe_mkdir(OUT_DIR)
rows = crawl_cela_isms(TARGET_URL)
print(f"[INFO] Extracted rows: {len(rows)}")
print(f"[INFO] table={sum(r.content_type=='table' for r in rows)}, "
      f"paragraph={sum(r.content_type=='paragraph' for r in rows)}, "
      f"list={sum(r.content_type=='list' for r in rows)}, "
      f"link={sum(r.content_type=='link' for r in rows)}")

csv_path = os.path.join(OUT_DIR, "cela_isms_1_1.csv")
json_path = os.path.join(OUT_DIR, "cela_isms_1_1.json")
save_as_csv(rows, csv_path)
save_as_json(rows, json_path)
print(f"[OK] Saved: {csv_path}")
print(f"[OK] Saved: {json_path}")

[INFO] Extracted rows: 292
[INFO] table=0, paragraph=195, list=25, link=72
[OK] Saved: ../data/cela_isms_1_1.csv
[OK] Saved: ../data/cela_isms_1_1.json


In [4]:
rows

[ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='정보보호 관리체계 (ISMS)', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='1.1.1. 경 영진의 참여', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='최고경영자는 정보보호 및 개인정보보호 관리체계의 수립과 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 체계를 수립하여 운영하여야 한다.', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='1. 정보보호 및 개인정보보호 관리체계의 수립 및 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 등의 책임과 역할을 문서화하고 있는가? (간편인증-7의2) (간편인증-7의3)', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='- 정보보호 및 개인정보보호 정책의 제· 개정, 위험관리, 내부감사 등 관리체계 운영의 중요 사안에 대하여 경영진이 참여할 수 있도록 활동의 근거를 정보보호 및 개인정보보호 정책 또는 시행문서에 명시', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='2. 경영진이 정보보호 및 개인정보보호 활동에 관한 의사결정에 적극적으로 참여할 수 있는 보고, 검토 및 승인 절차를 수립· 이행하고 있는가? (간편인증-7의2) (간편인증-7의

In [54]:
import re
from dataclasses import dataclass
from typing import List, Optional, Dict, Any

# ====== Input/Output Schema ======
@dataclass
class ContentRow:
    section_order: int
    section_title: str
    content_type: str
    text: str
    href: Optional[str] = None

@dataclass
class LawDoc:
    text: str
    meta: Dict[str, Any]

# ====== Utility ======
ARTICLE_RE = re.compile(r'^\s*(\d+(?:\.\d+)+)\.\s*(.+?)\s*$')  # e.g., "1.1.1. 경 영진의 참여"

# 이 라벨이 등장하면, 그 **다음 내용 전부**(다음 헤딩 전까지)를 삭제
REMOVE_LABELS = {
    "운영 내역(증적) 예시",
    "** 인증심사 결함사항 예시 **",
    "** 인증심사 결 함사항 예시 **",
    "** 인증심사 결함사항 예시 ** (개정 23.11.23.)",
}

def norm_space(s: str) -> str:
    return re.sub(r'\s+', ' ', s).strip()

def is_heading(text: str):
    m = ARTICLE_RE.match(text)
    if not m:
        return None
    art_id, title = m.group(1), norm_space(m.group(2))
    return art_id, title

def is_remove_label_line(text: str) -> bool:
    t = norm_space(text)
    if t in REMOVE_LABELS:
        return True
    # “인증심사 … 예시”류 변형 대응(혹시 다른 변형이 있어도)
    if "인증심사" in t and t.endswith("예시"):
        return True
    return False

INLINE_PATTERNS = [
    # (간편인증-7의2), (간편인증-7의3) 등 제거
    re.compile(r'\(간편인증-[^)]+\)'),
    # 기타 ‘(※ …)’ 단독 문장 제거는 라인 레벨에서 처리하지만,
    # 본문 중간 삽입된 경우도 방어적으로 제거
    re.compile(r'\(※[^)]+\)'),
]

def clean_inline_refs(line: str) -> str:
    s = line
    for pat in INLINE_PATTERNS:
        s = pat.sub('', s)
    # 중복 공백 정리
    return norm_space(s)

def safe_token_len(s: str) -> int:
    try:
        import tiktoken  # type: ignore
        enc = tiktoken.get_encoding("cl100k_base")
        return len(enc.encode(s))
    except Exception:
        return len(re.findall(r'\S+', s))

LAW_NAME_MAP = {
    "1.1": "관리체계 기반 마련"
    , "1.2": "위험 관리"
    , "1.3": "관리체계 운영"
    , "1.4": "관리체계 점검 및 개선"
    , "2.1": "정책, 조직, 자산 관리"
    , "2.2": "인적 보안"
    , "2.3": "외부자 보안"
    , "2.4": "물리 보안"
    , "2.5": "인증 및 권한 관리"
    , "2.6": "접근통제"
    , "2.7": "암호화 적용"
    , "2.8": "정보시스템 도입 및 개발 보안"
    , "2.9": "시스템 및 서비스 운영관리"
    , "2.10": "시스템 및 서비스 보안관리"
    , "2.11": "사고 예방 및 대응"
    , "2.12": "재해복구"
    , "3.1": "개인정보 수집 시 보호조치"
    , "3.2": "개인정보 보유 이용 시 보호조치"
    , "3.3": "개인정보 제공 시 보호조치"
    , "3.4": "개인정보 파기 시 보호조치"
    , "3.5": "정보주체 권리보호"
}

def infer_law_name(article_id: str) -> str:
    parts = article_id.split(".")
    for n in range(len(parts), 0, -1):
        key = ".".join(parts[:n])
        if key in LAW_NAME_MAP:
            return LAW_NAME_MAP[key]
    return "정보보호 관리체계"

# ====== Core Transform ======
def rows_to_lawdocs(
    rows: List[ContentRow],
    source_uri: str = "../data/isms.pdf",
    law_id: str = "isms_1_1",
    version: Optional[str] = None,
) -> List[LawDoc]:
    docs: List[LawDoc] = []

    cur_article_id: Optional[str] = None
    cur_article_title: Optional[str] = None
    cur_chunks: List[str] = []
    suppress_until_next_heading = False  # 라벨 이후 ~ 다음 헤딩 전까지 전체 무시

    def flush():
        if not cur_article_id or not cur_article_title:
            return
        law_name = infer_law_name(cur_article_id)
        header = f"{law_name} ({cur_article_title})"
        body = "\n".join(cur_chunks).strip()
        text = f"{header}\n {body}".strip()

        meta = {
            "law_id": law_id,
            "law_name": law_name,
            "article_id": cur_article_id,
            "article_title": cur_article_title,
            "effective_date": None,
            "tok_len": safe_token_len(text),
            "source_uri": source_uri,
            "version": version,
        }
        docs.append(LawDoc(text=text, meta=meta))

    for r in rows:
        raw = (r.text or "").strip()
        if not raw:
            continue

        # 새 조항 헤딩인지 먼저 확인 (라벨 억제 구간도 헤딩은 통과시켜야 함)
        h = is_heading(raw)
        if h:
            # 이전 기사 저장
            if cur_article_id:
                flush()
            cur_article_id, cur_article_title = h
            cur_chunks = []
            suppress_until_next_heading = False  # 새 헤딩에서 억제 해제
            continue

        # 억제 구간이면 무시
        if suppress_until_next_heading:
            continue

        # 제거 라벨을 만나면, 이후 전부 억제(다음 헤딩 나올 때까지)
        if is_remove_label_line(raw):
            suppress_until_next_heading = True
            continue

        # 단독 메타 라인(예: "(※ …)")은 제거
        if re.fullmatch(r'\(※.*\)', raw):
            continue

        # 인라인 표기 정리: (간편인증-7의2) 등 제거
        line = clean_inline_refs(raw)
        if not line:
            continue

        cur_chunks.append(line)

    if cur_article_id:
        flush()

    return docs


In [55]:
lawdocs = rows_to_lawdocs(rows, source_uri="../data/isms.pdf", law_id="isms_1_1")
# 필요 시 dict로 변환하여 확인

In [58]:
print(lawdocs[5].meta)

{'law_id': 'isms_1_1', 'law_name': '관리체계 기반 마련', 'article_id': '1.1.6', 'article_title': '자원 할당', 'effective_date': None, 'tok_len': 211, 'source_uri': '../data/isms.pdf', 'version': None}


In [59]:
crawl_list = {
    "1_1": "관리체계 기반 마련"
    , "1_2": "위험 관리"
    , "1_3": "관리체계 운영"
    , "1_4": "관리체계 점검 및 개선"
    , "2_1": "정책, 조직, 자산 관리"
    , "2_2": "인적 보안"
    , "2_3": "외부자 보안"
    , "2_4": "물리 보안"
    , "2_5": "인증 및 권한 관리"
    , "2_6": "접근통제"
    , "2_7": "암호화 적용"
    , "2_8": "정보시스템 도입 및 개발 보안"
    , "2_9": "시스템 및 서비스 운영관리"
    , "2_10": "시스템 및 서비스 보안관리"
    , "2_11": "사고 예방 및 대응"
    , "2_12": "재해복구"
    , "3_1": "개인정보 수집 시 보호조치"
    , "3_2": "개인정보 보유 이용 시 보호조치"
    , "3_3": "개인정보 제공 시 보호조치"
    , "3_4": "개인정보 파기 시 보호조치"
    , "3_5": "정보주체 권리보호"
}

In [ ]:
for i, v in crawl_list.items():
    
    # ===== 설정 =====
    TARGET_URL = f"https://www.cela.kr/ISMS_{i}"
    OUT_DIR = "../data"
    TIMEOUT = 20
    RETRY = 3
    SLEEP_SEC = 0.8
    USE_SELENIUM_FALLBACK = True   # 정적 파싱이 비면 Selenium로 재시도(+토글 전부 펼치기)
    
    HEADERS = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/134.0.0.0 Safari/537.36"
        )
    }
    
    safe_mkdir(OUT_DIR)
    rows = crawl_cela_isms(TARGET_URL)
    print(f"[INFO] Extracted rows: {len(rows)}")
    print(f"[INFO] table={sum(r.content_type=='table' for r in rows)}, "
          f"paragraph={sum(r.content_type=='paragraph' for r in rows)}, "
          f"list={sum(r.content_type=='list' for r in rows)}, "
          f"link={sum(r.content_type=='link' for r in rows)}")
    
    # csv_path = os.path.join(OUT_DIR, "cela_isms_1_1.csv")
    # json_path = os.path.join(OUT_DIR, "cela_isms_1_1.json")
    # save_as_csv(rows, csv_path)
    # save_as_json(rows, json_path)
    # print(f"[OK] Saved: {csv_path}")
    # print(f"[OK] Saved: {json_path}")
    
    lawdocs = rows_to_lawdocs(rows, source_uri="../data/isms.pdf", law_id=f"isms_{i}")
    # 필요 시 dict로 변환하여 확인

    lawdocs.append(lawdocs)

[INFO] Extracted rows: 292
[INFO] table=0, paragraph=195, list=25, link=72
[INFO] Extracted rows: 204
[INFO] table=0, paragraph=111, list=21, link=72


In [64]:
ret_lawdocs[5]

[LawDoc(text='인적 보안 (주요 직무자 지정 및 관리)\n 개인정보 및 중요정보의 취급이나 주요 시스템 접근 등 주요 직무의 기준과 관리방안을 수립하고, 주요 직무자를 최소한으로 지정하여 그 목록을 최신으로 관리하여야 한다.\n1. 개인정보 및 중요정보의 취급, 주요 시스템 접근 등 주요 직무의 기준을 명확히 정의하고 있는가?\n※ 주요 직무의 기준(예시)\n- 중요정보(개인정보, 인사정보, 영업비밀, 산업기밀, 재무정보 등) 취급\n- 중요 정보시스템(서버, 데이터베이스, 응용 프로그램 등) 및 개인정보처리시스템 운영· 관리\n- 정보보호 및 개인정보보호 관리 업무 수행\n- 보안시스템 운영 등\n(가상자산사업자) 월렛 및 개인키, 거래원장에 접근가능한 직무에 대하여 정의하고 있는가?\n- 월렛 및 개인키, 거래원장에 접근가능한 직무에 대하여 정의\n2. 주요 직무를 수행하는 임직원 및 외부자를 주요 직무자로 지정하고 그 목록을 최신으로 관리하고 있는가?\n- 주요 직무자 현황을 파악하여 주요 직무자로 공식 지정\n- 지정된 주요 직무자에 대하여 목록으로 관리\n- 주요 직무자의 신규 지정 및 변경, 해제 시 목록 업데이트\n- 정기적으로 주요 직무자 지정 현황 및 적정성을 검토하여 목록 최신화\n3. 업무상 개인정보를 취급하는 자를 개인정보취급자로 지정하고 목록을 최신으로 관리하고 있는가?\n- 업무상 개인정보를 처리하는 개인정보취급자에 대해서는 목록으로 관리\n- 개인정보취급자 목록에는 개인정보 처리업무에 대한 위탁을 받은 수탁자의 개인정보취급자도 포함 (다만 수탁자의 개인정보취급자 중 개인정보처리시스템에 접근권한이 없는 개인정보취급자에 대한 목록관리는 수탁자 자체적으로 관리 가능)\n- 정기적으로 개인정보취급자 지정 현황 및 적정성을 검토하여 목록 최신화\n※ 개인정보취급자의 정의\n- 임직원, 파견근로자, 시간제근로자 등 개인정보처리자의 지휘· 감독을 받아 개인정보를 처리하는 자\n4 . 업무 필요성에 따라 주요 직무자 및 개인정보취급자 지